In [8]:
from get_datasets import get_data
train_dataset, val_dataset, test_dataset, vocab = get_data('SentimentClassifier/')

In [12]:
from torch.utils.data import DataLoader

def sorted_collate(batch):
    x, y = zip(*batch)
    order = np.argsort([len(xi) for xi in x])[::-1]
    x = [torch.LongTensor(x[i]) for i in order]
    y = torch.stack([y[i] for i in order])[:, None]
    return x,y

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, collate_fn=sorted_collate)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, collate_fn=sorted_collate)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, collate_fn=sorted_collate)

In [15]:
from torch import jit
import torch
from torch import nn

class SkipGRUCell(jit.ScriptModule):
    def __init__(self, input_size, hidden_size):
        """Simple SkipGRU cell.
        Args:
            input_size (int): size of input space of GRU
            hidden_size (int): size of hidden space of GRU
        """
        super().__init__()
        self.inner_gru = nn.GRUCell(input_size, hidden_size)

    @jit.script_method
    def forward(self, input, hidden, mix):
        """
        Args:
            input (torch.FloatTensor): input vector for the current step
            hidden (torch.FloatTensor): hidden vector for the previous step
            mix (torch.FloatTensor): mixture vector which defines proportion of mix between previous step and current.
        Returns:
            hidden (torch.FloatTensor): hidden vetor for the current step
        """
        # type: (Tensor, Tensor, Tensor) -> Tensor

        output = self.inner_gru(input, hidden)
        output = output * mix + hidden * (1 - mix)

        return output

from typing import List, Tuple

class SkipGRULayer(jit.ScriptModule):
    __constants__ = ['hidden_size']
    def __init__(self, input_size, hidden_size):
        """
        Layer consisted of SkipGRU cells.
        Args:
            input_size (int): size of input space of GRU
            hidden_size (int): size of hidden space of GRU
        """
        super().__init__()
        self.cell = SkipGRUCell(input_size, hidden_size)
        self.hidden_size = hidden_size

    @jit.script_method
    def forward(self, input, mix):
        """
        Forward pass on SkipGRU layer.
        Args:
            input (torch.FloatTensor): inputs for the SkipGRU layer of form [BxTxN]
            mix (torch.FloatTensor): mixture coefficients for thehidden steps of form [BxTx1]
        Returns:
            h (torch.FloatTensor): history of outputs by steps. Size [BxTxH]
            o (torch.FloatTensor): last hidden step of size [BxH]
        """
        # type: (Tensor, Tensor) -> Tuple[Tensor, Tensor]
        hidden = torch.zeros(input.shape[0], self.hidden_size,
                             device=self.cell.inner_gru.weight_ih.device)
        inputs = input.unbind(1)
        mixtures = mix.unbind(1)
        outputs = jit.annotate(List[Tensor], [])
        for i in range(len(inputs)):
            hidden = self.cell(inputs[i], hidden, mixtures[i])
            outputs += [hidden]
        return torch.stack(outputs, 1), hidden


class SkipGRU(nn.Module):
    def __init__(self, input_size=None, hidden_size=None, layer=None, do_copy_weights=False):
        """
        SkipRNN layer with trivial binarizer (All betas are directly parametrized.)
        Args:
            input_size (int): size of input space for SkipGRU
            hidden_size (int): size of hidden space for SkipGRU
            layer (nn.GRU): original GRU layers to copy sizes and make a link to weights
        """
        super().__init__()
        if layer is not None:
            self.layer = SkipGRULayer(layer.input_size, layer.hidden_size)
            self.impute_weights(layer, do_copy_weights)
        elif (input_size is not None) and (hidden_size is not None):
            self.layer = SkipGRULayer(input_size, hidden_size)
        else:
            raise Exception('Either layer or input_size & hidden_size are required')

    def impute_weights(self, donor_layer, copy=False):
        """
        Get weights from instance of nn.GRU.
        Args:
            donor_layer (nn.GRU): pretrained layer to get weights from.
            copy (bool): if True, will make copy of weights instead of linkage.
        """
        assert isinstance(donor_layer, nn.GRU), 'Wrong type of donor layer. GRU required!'

        if copy:
            self.layer.cell.inner_gru.weight_ih.data = donor_layer.weight_ih_l0.clone()
            self.layer.cell.inner_gru.weight_hh.data = donor_layer.weight_hh_l0.clone()
            self.layer.cell.inner_gru.bias_ih.data = donor_layer.bias_ih_l0.clone()
            self.layer.cell.inner_gru.bias_hh.data = donor_layer.bias_hh_l0.clone()
        else:
            self.layer.cell.inner_gru.weight_ih = donor_layer.weight_ih_l0
            self.layer.cell.inner_gru.weight_hh = donor_layer.weight_hh_l0
            self.layer.cell.inner_gru.bias_ih = donor_layer.bias_ih_l0
            self.layer.cell.inner_gru.bias_hh = donor_layer.bias_hh_l0

    def forward(self, x, b, l=None):
        """
        Make forward SkipGRU pass.
        Args:
            x (torch.FloatTensor): input of size [BxTxN]. NB! sequences should be padded from the end, not from the start position.
            u (torch.FloatTensor): mixture coefficients of size [BxTx1]
            l (torch.LongTensor): lengths of the padded sequences. If not provided, output will contain last elements of sequences.
        Returns:
            h (torch.FloatTensor, nn.utils.rnn.PackedSequence): history of hidden states of size [BxTxH]
            o (torch.FloatTensor): last hidden state of size [BxH]
        """
        h, o = self.layer(x, b)
        if l is not None:
            o = h[torch.arange(h.shape[0]), l-1]
        return h, o

In [16]:
import torch
import numpy as np
from torch import nn
#from skiprnn import SkipGRU
import torch.functional as F

class IMDbRNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_classes):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = SkipGRU(embedding_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)
        self.emb = nn.Embedding(vocab_size, embedding_size)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
        x, l = self.prepare_sequence(x)
        #на данном этапе учим с маской из одних единиц
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        return self.cls(o)

In [18]:
device = torch.device('cpu')
model = IMDbRNN(2000, 32, 16, 1)
model.to(device)

IMDbRNN(
  (rnn): SkipGRU(
    (layer): SkipGRULayer(
      original_name=SkipGRULayer
      (cell): SkipGRUCell(
        original_name=SkipGRUCell
        (inner_gru): ScriptModule(original_name=GRUCell)
      )
    )
  )
  (cls): Linear(in_features=16, out_features=1, bias=True)
  (emb): Embedding(2000, 32)
)

In [21]:
import torch
import torch.nn as nn
from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import AUCCallback, F1ScoreCallback, CheckpointCallback, CriterionCallback, CriterionAggregatorCallback

/Users/ayunts/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [22]:
num_epochs = 100
logdir = "./logs/exp3"
loaders = {
    "train": train_loader,
    "valid": val_loader
}

criterion = {
   "bce": nn.BCEWithLogitsLoss()
}

optimizer = torch.optim.Adam([
    {'params': model.parameters(), 'lr': 3e-4}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner()

In [24]:
runner.train(
    model=model,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_bce",
                     criterion_key='bce', multiplier=1.),
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_bce']),
               AUCCallback(num_classes=1), 
                F1ScoreCallback(),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='auc/_mean',
    minimize_metric=False,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [26]:
state = torch.load("best_skiprnn.pth",map_location='cpu')

model.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

In [27]:
model.eval()

IMDbRNN(
  (rnn): SkipGRU(
    (layer): SkipGRULayer(
      original_name=SkipGRULayer
      (cell): SkipGRUCell(
        original_name=SkipGRUCell
        (inner_gru): ScriptModule(original_name=GRUCell)
      )
    )
  )
  (cls): Linear(in_features=16, out_features=1, bias=True)
  (emb): Embedding(2000, 32)
)

In [28]:
import numpy as np
from tqdm import tqdm

predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        predictions.append(model(x).detach().cpu().numpy())
        labels.append(y.detach().cpu().numpy())
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

100%|██████████| 25/25 [00:22<00:00,  1.58it/s]


In [29]:
from sklearn.metrics import roc_auc_score

roc_auc_score(labels, predictions)

0.7049218304

In [30]:
from binarizers import HintonBinarizer

In [31]:
hb = HintonBinarizer()

In [32]:
for x,y in train_loader:
    break

Запускаем для предложения x[0]

In [33]:
x[0].shape

torch.Size([1216])

In [34]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        
        #считаем с бинаризованной маской u
        b_new = torch.repeat_interleave(hb(self.u), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [35]:
model_bin = IMDbRNN_bin(x[0].shape[0])

Делаем датасет из предложения x[0] размножив его несколько раз

In [36]:
from torch.utils.data import Dataset as BaseDataset

class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)


In [37]:
num_epochs = 100  # change me
logdir = "./logs/exp5"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [38]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-01 16:08:13,436] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=5.5163 | _timers/batch_time=0.3638 | _timers/data_time=0.0003 | _timers/model_time=0.3635 | f1_score=8.167e-08 | loss=21.2579 | loss_dist=21.2157 | loss_mean=0.0422
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=9.0970 | _timers/batch_time=0.2243 | _timers/data_time=0.0003 | _timers/model_time=0.2240 | f1_score=8.382e-08 | loss=20.2173 | loss_dist=20.1747 | loss_mean=0.0426
[2019-12-01 16:08:13,436] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=5.5163 | _timers/batch_time=0.3638 | _timers/data_time=0.0003 | _timers/model_time=0.3635 | f1_score=8.167e-08 | loss=21.2579 | loss_dist=21.2157 | loss_mean=0.0422
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=9.0970 | _timers/batch_time=0.2243 | _timers/data_time=0.0003 | _timers/model_time=0.2240 | f1_score=8.382e-08 | loss=20.2173 | los

[2019-12-01 16:08:36,288] 
6/100 * Epoch 6 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.6621 | _timers/batch_time=0.3555 | _timers/data_time=0.0004 | _timers/model_time=0.3551 | f1_score=2.157e-07 | loss=5.9248 | loss_dist=5.8807 | loss_mean=0.0441
6/100 * Epoch 6 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=10.5428 | _timers/batch_time=0.1931 | _timers/data_time=0.0004 | _timers/model_time=0.1927 | f1_score=2.378e-07 | loss=5.4227 | loss_dist=5.3794 | loss_mean=0.0433
[2019-12-01 16:08:36,288] 
6/100 * Epoch 6 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.6621 | _timers/batch_time=0.3555 | _timers/data_time=0.0004 | _timers/model_time=0.3551 | f1_score=2.157e-07 | loss=5.9248 | loss_dist=5.8807 | loss_mean=0.0441
6/100 * Epoch 6 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=10.5428 | _timers/batch_time=0.1931 | _timers/data_time=0.0004 | _timers/model_time=0.1927 | f1_score=2.378e-07 | loss=5.4227 | loss_dis

[2019-12-01 16:08:58,912] 
11/100 * Epoch 11 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.6236 | _timers/batch_time=0.3563 | _timers/data_time=0.0003 | _timers/model_time=0.3560 | f1_score=5.056e-07 | loss=3.5650 | loss_dist=3.5204 | loss_mean=0.0446
11/100 * Epoch 11 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=10.4668 | _timers/batch_time=0.1950 | _timers/data_time=0.0003 | _timers/model_time=0.1946 | f1_score=4.944e-07 | loss=3.5825 | loss_dist=3.5382 | loss_mean=0.0443
[2019-12-01 16:09:03,264] 
12/100 * Epoch 12 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=5.8306 | _timers/batch_time=0.3439 | _timers/data_time=0.0003 | _timers/model_time=0.3435 | f1_score=5.018e-07 | loss=3.5708 | loss_dist=3.5267 | loss_mean=0.0441
12/100 * Epoch 12 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=10.5144 | _timers/batch_time=0.1949 | _timers/data_time=0.0003 | _timers/model_time=0.1946 | f1_score=6.292e-07 | loss=1.2781 | 

...

[2019-12-01 16:13:36,694] 
75/100 * Epoch 75 (train): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=5.9176 | _timers/batch_time=0.3393 | _timers/data_time=0.0003 | _timers/model_time=0.3389 | f1_score=7.840e-07 | loss=1.6852 | loss_dist=1.6267 | loss_mean=0.0585
75/100 * Epoch 75 (valid): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=10.6631 | _timers/batch_time=0.1911 | _timers/data_time=0.0003 | _timers/model_time=0.1908 | f1_score=7.840e-07 | loss=1.6852 | loss_dist=1.6267 | loss_mean=0.0585
[2019-12-01 16:13:41,033] 
76/100 * Epoch 76 (train): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=5.8784 | _timers/batch_time=0.3413 | _timers/data_time=0.0004 | _timers/model_time=0.3409 | f1_score=7.839e-07 | loss=1.6854 | loss_dist=1.6267 | loss_mean=0.0587
76/100 * Epoch 76 (valid): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=10.5899 | _timers/batch_time=0.1926 | _timers/data_time=0.0004 | _timers/model_time=0.1922 | f1_score=7.839e-07 | loss=1.6855 | 

[2019-12-01 16:14:02,705] 
81/100 * Epoch 81 (train): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=5.9001 | _timers/batch_time=0.3401 | _timers/data_time=0.0002 | _timers/model_time=0.3398 | f1_score=7.901e-07 | loss=1.6915 | loss_dist=1.6332 | loss_mean=0.0582
81/100 * Epoch 81 (valid): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=10.6000 | _timers/batch_time=0.1930 | _timers/data_time=0.0003 | _timers/model_time=0.1927 | f1_score=7.901e-07 | loss=1.6915 | loss_dist=1.6332 | loss_mean=0.0582
[2019-12-01 16:14:02,705] 
81/100 * Epoch 81 (train): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=5.9001 | _timers/batch_time=0.3401 | _timers/data_time=0.0002 | _timers/model_time=0.3398 | f1_score=7.901e-07 | loss=1.6915 | loss_dist=1.6332 | loss_mean=0.0582
81/100 * Epoch 81 (valid): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=10.6000 | _timers/batch_time=0.1930 | _timers/data_time=0.0003 | _timers/model_time=0.1927 | f1_score=7.901e-07 | loss=1.6915 | 

[2019-12-01 16:14:24,297] 
86/100 * Epoch 86 (train): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=5.9862 | _timers/batch_time=0.3351 | _timers/data_time=0.0003 | _timers/model_time=0.3348 | f1_score=7.940e-07 | loss=1.6958 | loss_dist=1.6373 | loss_mean=0.0585
86/100 * Epoch 86 (valid): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=10.4578 | _timers/batch_time=0.1952 | _timers/data_time=0.0003 | _timers/model_time=0.1948 | f1_score=7.940e-07 | loss=1.6958 | loss_dist=1.6373 | loss_mean=0.0585
[2019-12-01 16:14:24,297] 
86/100 * Epoch 86 (train): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=5.9862 | _timers/batch_time=0.3351 | _timers/data_time=0.0003 | _timers/model_time=0.3348 | f1_score=7.940e-07 | loss=1.6958 | loss_dist=1.6373 | loss_mean=0.0585
86/100 * Epoch 86 (valid): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=10.4578 | _timers/batch_time=0.1952 | _timers/data_time=0.0003 | _timers/model_time=0.1948 | f1_score=7.940e-07 | loss=1.6958 | 

[2019-12-01 16:14:45,931] 
91/100 * Epoch 91 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=5.8970 | _timers/batch_time=0.3401 | _timers/data_time=0.0002 | _timers/model_time=0.3398 | f1_score=7.939e-07 | loss=1.6967 | loss_dist=1.6372 | loss_mean=0.0595
91/100 * Epoch 91 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=10.5226 | _timers/batch_time=0.1936 | _timers/data_time=0.0003 | _timers/model_time=0.1932 | f1_score=7.939e-07 | loss=1.6971 | loss_dist=1.6372 | loss_mean=0.0599
[2019-12-01 16:14:50,241] 
92/100 * Epoch 92 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=5.8553 | _timers/batch_time=0.3424 | _timers/data_time=0.0002 | _timers/model_time=0.3422 | f1_score=7.940e-07 | loss=1.6974 | loss_dist=1.6372 | loss_mean=0.0601
92/100 * Epoch 92 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=10.5564 | _timers/batch_time=0.1933 | _timers/data_time=0.0004 | _timers/model_time=0.1929 | f1_score=7.941e-07 | loss=1.6981 | 

[2019-12-01 16:15:11,876] 
97/100 * Epoch 97 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=5.8950 | _timers/batch_time=0.3403 | _timers/data_time=0.0003 | _timers/model_time=0.3400 | f1_score=8.008e-07 | loss=1.6999 | loss_dist=1.6392 | loss_mean=0.0607
97/100 * Epoch 97 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=10.4314 | _timers/batch_time=0.1950 | _timers/data_time=0.0002 | _timers/model_time=0.1948 | f1_score=8.019e-07 | loss=1.7010 | loss_dist=1.6402 | loss_mean=0.0608
[2019-12-01 16:15:11,876] 
97/100 * Epoch 97 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=5.8950 | _timers/batch_time=0.3403 | _timers/data_time=0.0003 | _timers/model_time=0.3400 | f1_score=8.008e-07 | loss=1.6999 | loss_dist=1.6392 | loss_mean=0.0607
97/100 * Epoch 97 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=10.4314 | _timers/batch_time=0.1950 | _timers/data_time=0.0002 | _timers/model_time=0.1948 | f1_score=8.019e-07 | loss=1.7010 | 

In [40]:
from binarizers import ConcreteBinarizer

In [41]:
cb = ConcreteBinarizer()

In [44]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        
        #считаем с бинаризованной маской u
        b_new = torch.repeat_interleave(cb(self.u), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [45]:
model_bin = IMDbRNN_bin(x[0].shape[0])

In [46]:
from torch.utils.data import Dataset as BaseDataset

class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)



In [47]:
num_epochs = 100  # change me
logdir = "./logs/exp5"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [ ]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>